#1 Introducción

#2 Armado del ambiente

In [ ]:
! mpicc -o hello hello.c

     |████████████████████████████████| 1.4MB 17.2MB/s 
  Created wheel for mpi4py: filename=mpi4py-3.0.3-cp36-cp36m-linux_x86_64.whl size=2074485 sha256=16b46c33809961f6bd2fd65d41011207750a99a3b9385868f7f748b0d8a3bb68
  Stored in directory: /root/.cache/pip/wheels/18/e0/86/2b713dd512199096012ceca61429e12b960888de59818871d6
Successfully built mpi4py


In [ ]:


codigo = """
#include <string.h>
#include <stdio.h>
#include <mpi.h>

int main(int argc, char** argv)
{
  int rank;
  int cantidadProcesos;
  int tag = 1;
  char mensaje[100];
  MPI_Status;

  MPI_Init(NULL,NULL);

  MPI_Comm_rank(MPI_COMM_WORLD,&rank);
  MPI_Comm_size(MPI_COMM_WORLD,&cantidadProcesos);

  if(rank!=0)
  {
    int dest = 0;
    sprintf(mensaje,"hola mundo soy el thread %d de %d,rank,cantidadProcesos);
    MPI_Send(&mensaje,strlen(mensaje)+1,MPI_CHAR,dest,tag,MPI_COMM_WORLD);
  }
  else
  {
    for(int i=1;i<cantidadProcesos;i++)
    {
      MPI_Recv(&mensaje,100,MPI_CHAR,i,tag,MPI_COMM_WORLD,&status);
      printf("%s\n",mensaje);
    }
  }

  MPI_Finalize();
}

"""

file = open('hello.py', 'w')
file.write(codigo)
file.close()

    ! mpiexec --allow-run-as-root -np 3 python hello.py hola

parametros:  ['hello.py', 'hola']
soy el proceso:  2 no hago nada soy un inutil
parametros:  ['hello.py', 'hola']
soy el proceso:  1 multiplico a*b: 2
parametros:  ['hello.py', 'hola']
soy el proceso:  0 sumo a+b: 3


#3 Desarrollo

#4 Tabla de pasos

#5 Conclusiones

#6 Bibliografia